<a href="https://colab.research.google.com/github/nicoredo/Extension/blob/main/Spcay%20funcionante%20pero%20no%20util.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install spacy

In [2]:
!python -m spacy download es_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 56.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [7]:
import spacy
import json

# Cargar el modelo de spaCy en español
nlp = spacy.load("es_core_news_sm")

# Cargar los criterios de estudios desde el archivo JSON
with open('criterios_estudios.json', 'r') as f:
    criterios_estudios = json.load(f)

# Cargar los sinónimos desde el archivo JSON
with open('terminologia_medica.json', 'r') as f:
    sinonimos = json.load(f)

# Historia clínica de ejemplo
historia_clinica = "Pac 70a, diabetico medicado con dbi, erc con clearence de 70 y rac 45, obeso."

# Función para evaluar los criterios de inclusión con sinónimos
def evaluar_criterios_con_sinonimos(historia, criterios, sinonimos):
    faltantes = []

    # Procesar la historia clínica con spaCy
    doc = nlp(historia)

    # Evaluar cada criterio de inclusión
    for categoria, valores in criterios.items():
        if categoria == 'edad':
            # Buscar la edad en el texto
            edad_encontrada = None
            for token in doc:
                if token.like_num:
                    edad_encontrada = int(token.text)
                    break
            if edad_encontrada is None or not any(eval(str(edad_encontrada) + valor) for valor in valores):
                faltantes.append(f"edad {valores[0]}")
        else:
            # Buscar otros criterios en el texto completo
            found = False
            for valor in valores:
                # Verificar sinónimos
                for sinos in sinonimos.values():
                    if valor.lower() in sinos:
                        if any(sinonimo.lower() in historia.lower() for sinonimo in sinos[valor.lower()]):
                            found = True
                            break
                if found:
                    break
            if not found:
                faltantes.append(valor)

    return faltantes

# Evaluar los criterios de inclusión para todos los estudios
resultados = {}
for nombre_estudio, detalles in criterios_estudios['estudios'].items():
    criterios_inclusion = detalles['inclusion']
    faltantes = evaluar_criterios_con_sinonimos(historia_clinica, criterios_inclusion, sinonimos)
    if not faltantes:
        resultados[nombre_estudio] = "El paciente cumple con los criterios de inclusión."
    else:
        resultados[nombre_estudio] = f"El paciente no cumple con los siguientes criterios: {', '.join(faltantes)}."

# Mostrar los resultados
for nombre_estudio, resultado in resultados.items():
    print(f"{nombre_estudio}: {resultado}")


LIBREXIA SCA - Milvexian: El paciente no cumple con los siguientes criterios: infarto agudo de miocardio, Troponina = +.
GZQO - Retatrutida (GLP-1R y GCGR): El paciente no cumple con los siguientes criterios: RAC >300.
HERMES - Ziltivekimab: El paciente no cumple con los siguientes criterios: insuficiencia cardiaca, pro bnp>300.
BAX-DUO (ARTIC): Baxdrostat/Dapa vs Dapaglifozina: El paciente no cumple con los siguientes criterios: hipertension arterial, telmisartan, RAC <5000.
BAX-DUO (PREVENT HF): Baxdrostat/Dapa vs Dapaglifozina: El paciente no cumple con los siguientes criterios: telmisartan, RAC <5000.
AROAPOC3-3004 - SHASTA: Plozasirán: El paciente no cumple con los siguientes criterios: hipertrigliceridemia, colesterol LDL < 130.
J3L-MC-EZEF: Lepodisiran: El paciente no cumple con los siguientes criterios: trigliceridos >175.
BalanceD-HF: Balcinrenona/Dapa vs Dapa: El paciente no cumple con los siguientes criterios: insuficiencia cardiaca, trigliceridos >175.
